In [1]:
import collections
import json
import os

from absl import app
from absl import flags

import numpy as np
import pandas as pd
import tensorflow as tf
import tqdm

2024-07-15 09:12:27.317609: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 09:12:27.703354: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 09:12:27.705378: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 09:12:29.003239: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
input_path = "/home/pbr-student/personal/thesis/jrdb/train_dataset_with_activity" #/labels/labels_2d_pose_stitched_coco/"
scene = "clark-center-2019-02-28_0"

In [5]:
# from human scene transformer, data subfolder
def get_file_handle(path, mode='rt'):
  file_handle = open(path, mode)
  return file_handle
scene_data_file = get_file_handle(
      os.path.join(input_path, 'labels', 'labels_3d', scene + '.json')
  )

# get agents_dict_from_detections
scene_data = json.load(scene_data_file)
agents = collections.defaultdict(list)
for frame in scene_data['labels']:
    ts = int(frame.split('.')[0])
    for det in scene_data['labels'][frame]:
      agents[det['label_id']].append((ts, det))
agents_dict = agents

In [25]:
# get_agents_features_with_box
max_distance_to_robot = 10
agents_pos_dict = collections.defaultdict(dict)
for agent_id, agent_data in agents_dict.items():
    for (ts, agent_instance) in agent_data:
        if agent_instance['attributes']['distance'] <= max_distance_to_robot:
            agents_pos_dict[(ts, agent_id)] = {
                'timestamp:': ts,
                'agent_id': agent_id,
                'p': np.array([agent_instance['box']['cx'],
                                agent_instance['box']['cy'],
                                agent_instance['box']['cz']]),
                # rotation angle is relative to negatiev x axis of robot
                'yaw': np.pi - agent_instance['box']['rot_z'],
                'l': agent_instance['box']['l'],
                'w': agent_instance['box']['w'],
                'h': agent_instance['box']['h']
            }
agents_features = agents_pos_dict

In [26]:
agents_df = pd.DataFrame.from_dict(
        agents_features, orient='index'
    ).rename_axis(['timestep', 'id']) 

In [154]:
## get agents 2d
# from human scene transformer, data subfolder
def get_file_handle(path, mode='rt'):
  file_handle = open(path, mode)
  return file_handle
scene_data_file = get_file_handle(
      os.path.join(input_path, 'labels', 'labels_2d_pose_coco', scene + '_image2.json')
  )

# get agents_dict_from_detections
scene_data = json.load(scene_data_file)
agents = collections.defaultdict(list)
for frame in scene_data['labels']:
    ts = int(frame.split('.')[0])
    for det in scene_data['labels'][frame]:
      agents[det['label_id']].append((ts, det))
agents_dict = agents
# get_agents_features_with_box
max_distance_to_robot = 10
agents_pos_dict = collections.defaultdict(dict)
for agent_id, agent_data in agents_dict.items():
    for (ts, agent_instance) in agent_data:
        if agent_instance['attributes']['distance'] <= max_distance_to_robot:
            agents_pos_dict[(ts, agent_id)] = {
                'p': np.array([agent_instance['box']['cx'],
                                agent_instance['box']['cy'],
                                agent_instance['box']['cz']]),
                # rotation angle is relative to negatiev x axis of robot
                'yaw': np.pi - agent_instance['box']['rot_z'],
                'l': agent_instance['box']['l'],
                'w': agent_instance['box']['w'],
                'h': agent_instance['box']['h']
            }
agents_features = agents_pos_dict

KeyError: 'labels'

In [170]:
# get agents keypoints
keypoint_path = os.path.join(input_path, 'labels', 'labels_2d_pose_coco')
scene_data_file = get_file_handle(os.path.join(keypoint_path, scene + '_image0.json'))
scene_data = json.load(scene_data_file)

agents_keypoints = collections.defaultdict(dict)
image_keypoints = collections.defaultdict(dict)

for frame in scene_data['annotations']:
    agents_keypoints[frame['id']] = {
        'keypoints': np.array(frame['keypoints']).reshape(17, 3),
        'track_id': frame['track_id'],
        'timestep': frame['image_id']}
for frame in scene_data['images']:
    image_keypoints[frame['id']] = {
        'timestep': frame['id'],
        'file_name': frame['file_name']
    }

In [171]:
keypoints_df = pd.DataFrame.from_dict(
        agents_keypoints, orient='index'
    )
keypoints_df

,keypoints,track_id,timestep
5790,"[[595.15, 127.65, 2.0], [594.85, 136.8, 2.0], ...",2,2
5791,"[[634.53, 121.84, 2.0], [633.75, 130.3, 2.0], ...",2,4
5792,"[[49.25, 155.1, 2.0], [49.74, 163.23, 2.0], [5...",7,4
5793,"[[670.53, 119.44, 2.0], [670.3, 127.95, 2.0], ...",2,6
5794,"[[79.05, 156.55, 2.0], [79.1, 164.25, 2.0], [8...",7,6
...,...,...,...
5696,"[[192.8, 134.3, 2.0], [194.18, 141.62, 0.0], [...",23,531
5719,"[[195.1, 135.0, 2.0], [195.74, 142.48, 0.0], [...",23,533
5742,"[[198.0, 134.6, 2.0], [198.6, 141.5, 0.0], [19...",23,535
5765,"[[201.0, 135.1, 2.0], [201.9, 142.5, 0.0], [19...",23,537


In [172]:
image_df = pd.DataFrame.from_dict(
        image_keypoints, orient='index'
    )
image_df

,timestep,file_name
1,1,image_0/clark-center-2019-02-28_0/000000.jpg
2,2,image_0/clark-center-2019-02-28_0/000001.jpg
3,3,image_0/clark-center-2019-02-28_0/000002.jpg
4,4,image_0/clark-center-2019-02-28_0/000003.jpg
5,5,image_0/clark-center-2019-02-28_0/000004.jpg
...,...,...
573,573,image_0/clark-center-2019-02-28_0/000572.jpg
574,574,image_0/clark-center-2019-02-28_0/000573.jpg
575,575,image_0/clark-center-2019-02-28_0/000574.jpg
576,576,image_0/clark-center-2019-02-28_0/000575.jpg


In [173]:
result = pd.merge(keypoints_df, image_df, on="timestep")
result

,keypoints,track_id,timestep,file_name
0,"[[595.15, 127.65, 2.0], [594.85, 136.8, 2.0], ...",2,2,image_0/clark-center-2019-02-28_0/000001.jpg
1,"[[634.53, 121.84, 2.0], [633.75, 130.3, 2.0], ...",2,4,image_0/clark-center-2019-02-28_0/000003.jpg
2,"[[49.25, 155.1, 2.0], [49.74, 163.23, 2.0], [5...",7,4,image_0/clark-center-2019-02-28_0/000003.jpg
3,"[[670.53, 119.44, 2.0], [670.3, 127.95, 2.0], ...",2,6,image_0/clark-center-2019-02-28_0/000005.jpg
4,"[[79.05, 156.55, 2.0], [79.1, 164.25, 2.0], [8...",7,6,image_0/clark-center-2019-02-28_0/000005.jpg
...,...,...,...,...
782,"[[192.8, 134.3, 2.0], [194.18, 141.62, 0.0], [...",23,531,image_0/clark-center-2019-02-28_0/000530.jpg
783,"[[195.1, 135.0, 2.0], [195.74, 142.48, 0.0], [...",23,533,image_0/clark-center-2019-02-28_0/000532.jpg
784,"[[198.0, 134.6, 2.0], [198.6, 141.5, 0.0], [19...",23,535,image_0/clark-center-2019-02-28_0/000534.jpg
785,"[[201.0, 135.1, 2.0], [201.9, 142.5, 0.0], [19...",23,537,image_0/clark-center-2019-02-28_0/000536.jpg


In [159]:
grouped_df1 = result.groupby(result['track_id'], as_index=False)['timestep'].apply(list)
grouped_df2 = result.groupby(result['track_id'], as_index=False)['file_name'].apply(list)
grouped_df3 = result.groupby(result['track_id'], as_index=False)['keypoints'].apply(list)
result12 = pd.merge(grouped_df1, grouped_df2, on="track_id")
grouped_result = pd.merge(result12, grouped_df3, on="track_id")
print(grouped_result)

    track_id                                           timestep  \
0          2  [4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, ...   
1          7  [56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 7...   
2          9                              [2, 4, 6, 1, 3, 5, 7]   
3         11  [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 2...   
4         18  [134, 136, 138, 140, 142, 144, 146, 148, 150, ...   
5         19  [44, 46, 48, 50, 52, 80, 82, 84, 86, 88, 90, 9...   
6         20  [50, 52, 64, 66, 68, 70, 72, 74, 76, 78, 80, 8...   
7         28  [10, 12, 14, 16, 18, 20, 22, 24, 32, 34, 36, 3...   
8         30  [88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 1...   
9         31  [200, 202, 204, 206, 208, 210, 212, 214, 216, ...   
10        32  [224, 226, 228, 230, 232, 234, 223, 225, 227, ...   
11        33  [220, 222, 234, 236, 238, 240, 219, 221, 223, ...   
12        49  [140, 142, 144, 146, 148, 150, 152, 178, 180, ...   
13        63  [94, 104, 106, 108, 110, 112, 114, 116, 118, 1..

In [160]:
grouped_result.iloc[0]

track_id                                                     2
timestep     [4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, ...
file_name    [image_2/clark-center-2019-02-28_0/000003.jpg,...
keypoints    [[[45.57, 119.6, 2.0], [43.55, 129.8, 2.0], [4...
Name: 0, dtype: object

In [161]:
agents_df

,,timestamp:,agent_id,p,yaw,l,w,h
timestep,id,,,,,,,
0,pedestrian:9,0,pedestrian:9,"[4.74, -5.97, -0.00499]",-0.831997,0.89,0.51,1.85003
1,pedestrian:9,1,pedestrian:9,"[4.55275, -6.13846, -0.00499]",-0.900197,0.89,0.51,1.85003
2,pedestrian:9,2,pedestrian:9,"[4.36551, -6.30692, -0.00499]",-0.968397,0.89,0.51,1.85003
3,pedestrian:9,3,pedestrian:9,"[4.18276, -6.45846, -0.00499]",-1.011557,0.89,0.51,1.85003
4,pedestrian:9,4,pedestrian:9,"[4.0, -6.61, -0.00499]",-1.054707,0.89,0.51,1.85003
...,...,...,...,...,...,...,...,...
573,pedestrian:71,573,pedestrian:71,"[-6.99793, 5.45091, -0.12345]",2.450693,0.93,0.50,1.77310
574,pedestrian:71,574,pedestrian:71,"[-7.06738, 5.50448, -0.12346]",2.451823,0.93,0.50,1.77310
575,pedestrian:71,575,pedestrian:71,"[-7.13684, 5.55804, -0.12345]",2.452953,0.93,0.50,1.77310


In [162]:
grouped_df2_1 = agents_df.groupby(agents_df['agent_id'], as_index=False)['timestamp:'].apply(list)
grouped_df2_1 = grouped_df2_1.rename(columns={'agent_id': 'agent_id', 'timestamp:': 'timestep'})
grouped_df2_2 = agents_df.groupby(agents_df['agent_id'], as_index=False)['p'].apply(list)
grouped_result2 = pd.merge(grouped_df2_1, grouped_df2_2, on="agent_id")
print(grouped_result2)

          agent_id                                           timestep  \
0   pedestrian:108  [241, 242, 243, 244, 245, 246, 247, 248, 249, ...   
1    pedestrian:11  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
2   pedestrian:110  [241, 242, 243, 244, 245, 246, 247, 248, 249, ...   
3   pedestrian:116  [263, 264, 265, 266, 267, 268, 269, 270, 271, ...   
4   pedestrian:117  [263, 264, 265, 266, 267, 268, 269, 270, 271, ...   
5    pedestrian:13  [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...   
6    pedestrian:15  [283, 284, 285, 286, 287, 288, 289, 290, 291, ...   
7   pedestrian:154  [253, 254, 255, 256, 257, 258, 259, 260, 261, ...   
8    pedestrian:18  [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 4...   
9    pedestrian:19  [23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3...   
10    pedestrian:2  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
11   pedestrian:20  [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3...   
12   pedestrian:21  [98, 99, 100, 101, 102, 103, 10

In [165]:
# merge 2d and 3d information together by tracks of timestamps
track_dict = {}
for idx in range(grouped_result.shape[0]):
    tmp = grouped_result.iloc[idx]['timestep']
    tmp_id = grouped_result.iloc[idx]['track_id']
    print(tmp_id)
    tmp.sort()
    for index, row in grouped_result2.iterrows():
        if int(row['agent_id'][11:]) == int(tmp_id):
            #print(row['timestep'])
            # for the same track get the corresponding indices where the persons are detected at the same time
            indices1 = np.argwhere(np.isin(np.asarray(row['timestep']),np.asarray(tmp)))
            indices2 = np.argwhere(np.isin(np.asarray(tmp), np.asarray(row['timestep'])))
            # filter all relevant features by the indices
            if len(indices1) >= 15:
                positions = np.asarray(row['p'])[indices1.squeeze()]
                keypoints = np.asarray(grouped_result.iloc[idx]['keypoints'])[indices2.squeeze()]
                track_dict[idx] = {
                    'positions': positions,
                    'poses': keypoints
                }
            break


2
7
9
11
18
19
20
28
30
31
32
33
49
63
69
71
72
73
116
117


In [164]:
track_df = pd.DataFrame.from_dict(
        track_dict, orient='index'
    )
track_df

,positions,poses
0,"[[4.13194, -2.95542, 0.07721], [3.87808, -3.04...","[[[45.57, 119.6, 2.0], [43.55, 129.8, 2.0], [4..."
1,"[[2.50859, -1.81368, 0.09151], [2.4551, -1.911...","[[[72.9, 71.65, 2.0], [75.0, 90.65, 1.0], [68...."
3,"[[0.72629, -5.2902, -0.01015], [0.52258, -5.30...","[[[458.27, 153.01, 2.0], [458.07, 161.32, 0.0]..."
4,"[[3.71883, -2.42315, 0.07215], [3.65019, -2.45...","[[[29.01, 113.14, 2.0], [27.3, 127.34, 2.0], [..."
5,"[[6.81584, -4.18835, 0.08889], [6.72486, -4.21...","[[[15.99, 163.4, 2.0], [12.26, 169.31, 2.0], [..."
6,"[[5.80452, -3.58044, 0.11722], [5.68905, -3.60...","[[[23.45, 155.25, 2.0], [21.05, 162.65, 2.0], ..."
7,"[[-0.18275, -6.1492, 0.03997], [-0.24967, -6.1...","[[[527.3, 145.1, 2.0], [525.75, 155.0, 2.0], [..."
8,"[[4.91, -3.03, 0.11765], [4.63117, -3.04057, 0...","[[[38.9, 133.05, 2.0], [41.15, 143.25, 2.0], [..."
9,"[[2.04152, -1.6117, 0.01541], [1.85965, -1.566...","[[[68.9, 70.0, 2.0], [72.2, 92.05, 2.0], [77.7..."
12,"[[5.00892, -4.43708, 1.15285], [4.9008, -4.429...","[[[101.0, 76.5, 2.0], [98.35, 84.1, 2.0], [103..."


# absolute positions to relative distances

In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
df = pd.read_pickle("df_jrdb_augmented.pkl") 
df

,positions,poses
0,"[[1.6433, 1.31074, 0.05498], [1.61661, 1.27148...","[[[21.4, 9.8, 2.0], [25.55, 31.25, 2.0], [38.1..."
1,"[[0.83958, -0.50863, 0.045], [0.85848, -0.4422...","[[[563.57, -178.91, 0.0], [587.46, -136.51, 0...."
2,"[[1.53163, 1.33974, 0.03798], [1.52811, 1.3346...","[[[11.24, 23.85, 2.0], [-2.52, 46.95, 0.0], [-..."
3,"[[1.84136, 1.52542, -0.005], [1.83469, 1.50237...","[[[60.4, 70.02, 2.0], [54.05, 88.92, 2.0], [58..."
4,"[[1.72412, 1.30398, -0.01211], [1.69748, 1.244...","[[[68.25, 39.05, 2.0], [69.55, 66.05, 2.0], [7..."
...,...,...
4289,"[[5.968267727453773, 5.490492822033447, -0.065...","[[[-354.86317850622595, 63.36005871571147, 1.0..."
4290,"[[-9.463164342874244, 4.724265857019843, 0.025...","[[[72.55889816272631, 74.74382113199131, 2.0],..."
4291,"[[-3.364940896402999, -0.649425695671161, -0.0...","[[[-135.90354242194445, 721.5244311575088, 2.0..."
4292,"[[-0.497088846780676, 2.0076951175310342, -0.1...","[[[558.0353499688866, -232.6153921929978, 1.0]..."


In [4]:
df.iloc[0]["positions"].shape

(43, 3)